# Feature selection

Feature selection is the process of selecting a subset of relevant features (variables, predictors) for use in model construction. Feature selection techniques are used for several reasons: to simplify models to make them easier to interpret, to reduce training time, to avoid the curse of dimensionality, to improve generalization by reducing overfitting (formally, variance reduction), and others.

Skforecast is compatible with the [feature selection methods implemented in the scikit-learn](https://scikit-learn.org/stable/modules/feature_selection.html) library. There are several methods for feature selection, but the most common are: 


**Recursive feature elimination**

Given an external estimator that assigns weights to features (e.g., the coefficients of a linear model), the goal of recursive feature elimination ([RFE](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html)) is to select features by recursively considering smaller and smaller sets of features. First, the estimator is trained on the initial set of features, and the importance of each feature is obtained either by a specific attribute (such as `coef_`, `feature_importances_`) or by a `callable`. Then, the least important features are pruned from the current set of features. This procedure is repeated recursively on the pruned set until the desired number of features to select is eventually reached. [RFECV](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFECV.html#sklearn.feature_selection.RFECV) performs RFE in a cross-validation loop to find the optimal number of features.

**Sequential Feature Selection**

Sequential Feature Selection ([SFS](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SequentialFeatureSelector.html)) can be either forward or backward, with the `direction` parameter controlling whether forward or backward SFS is used.

+ **Forward-SFS** is a greedy procedure that iteratively finds the best new feature to add to the set of selected features. It starts with zero features and finds the one that maximizes a cross-validated score when an estimator is trained on that single feature. Once this first feature is selected, the procedure is repeated, adding one new feature to the set of selected features. The procedure stops when the desired number of selected features is reached, as determined by the `n_features_to_select` parameter.

+ **Backward-SFS** follows the same idea, but works in the opposite direction. Instead of starting with no features and greedily adding features, it starts with all features and greedily removes features from the set. 

In general, forward and backward selection do not produce equivalent results. Also, one can be much faster than the other depending on the requested number of selected features: if we have 10 features and ask for 7 selected features, forward selection would need to perform 7 iterations while backward selection would only need to perform 3.

SFS differs does not require the underlying model to expose a `coef_` or `feature_importances_` attribute. However, it may be slower compared to the other approaches, considering that more models have to be evaluated. For example in backward selection, the iteration going from $m$ features to $m - 1$ features using k-fold cross-validation requires fitting $m * k$ models to be evaluated.

For example in backward selection, the iteration going from $n$ features to $n - m$ features using k-fold cross-validation requires fitting $m * k$ models to be evaluated.

**Feature selection based on threshold (SelectFromModel)**

[SelectFromModel](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectFromModel.html) can be used along with any estimator that has a `coef_` or `feature_importances_` attribute after fitting. Features are considered unimportant and removed, if the corresponding `coef_` or `feature_importances_` values are below the given `threshold` parameter. In addition to specifying the `threshold` numerically, there are built-in heuristics for finding a threshold using a string argument. Available heuristics are "mean", "median" and float multiples of these, such as "0.1\*mean".

This method is very fast compared to the others because it does not require any additional model training. However, it does not evaluate the impact of feature removal on the model. It is often used for an initial selection before applying another more computationally expensive feature selection method.

<div class="admonition note" name="html-admonition" style="background: rgba(0,191,191,.1); padding-top: 0px; padding-bottom: 6px; border-radius: 8px; border-left: 8px solid #00bfa5; border-color: #00bfa5; padding-left: 10px; padding-right: 10px;">

<p class="title">
    <i style="font-size: 18px; color:#00bfa5;"></i>
    <b style="color: #00bfa5;">&#128161 Tip</b>
</p>

Feature selection is a powerful tool for improving the performance of machine learning models. However, it is computationally expensive and can be time-consuming. Since the goal is to find the best subset of features, not the best model, it is not necessary to use the entire data set or a highly complex model. Instead, it is recommended to use a <b>small subset of the data and a simple model</b>. Once the best subset of features has been identified, the model can then be trained using the entire dataset and a more complex configuration.
<br><br>
For example, in this use case, the model is an <code>LGMBRegressor</code> with 900 trees and a maximum depth of 7. However, to find the best subset of features, only 100 trees and a maximum depth of 5 are used.

</div>


<div class="admonition note" name="html-admonition" style="background: rgba(255,145,0,.1); padding-top: 0px; padding-bottom: 6px; border-radius: 8px; border-left: 8px solid #ff9100; border-color: #ff9100; padding-left: 10px; padding-right: 10px">

<p class="title">
    <i style="font-size: 18px; color:#ff9100; border-color: #ff1744;"></i>
    <b style="color: #ff9100;"> <span style="color: #ff9100;">&#9888;</span> Warning</b>
</p>

The current version of skforecast only supports feature selection for forecasters of type <code>ForecasterAutoreg</code> and <code>ForecasterAutoregCustom</code>.

</div>

## Libraries

In [1]:
# Libraries
# ==============================================================================
import numpy as np
from lightgbm import LGBMRegressor
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import ShuffleSplit
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import select_features
from skforecast.datasets import fetch_dataset

## Data

In [2]:
# Download data
# ==============================================================================
data = fetch_dataset(name="bike_sharing_extended_features")
data.head(3)

bike_sharing_extended_features
------------------------------
Hourly usage of the bike share system in the city of Washington D.C. during the
years 2011 and 2012. In addition to the number of users per hour, the dataset
was enriched by introducing supplementary features. Addition includes calendar-
based variables (day of the week, hour of the day, month, etc.), indicators for
sunlight, incorporation of rolling temperature averages, and the creation of
polynomial features generated from variable pairs. All cyclic variables are
encoded using sine and cosine functions to ensure accurate representation.
Fanaee-T,Hadi. (2013). Bike Sharing Dataset. UCI Machine Learning Repository.
https://doi.org/10.24432/C5W894.
Shape of the dataset: (17352, 90)


,users,weather,month_sin,month_cos,week_of_year_sin,week_of_year_cos,week_day_sin,week_day_cos,hour_day_sin,hour_day_cos,...,temp_roll_mean_1_day,temp_roll_mean_7_day,temp_roll_max_1_day,temp_roll_min_1_day,temp_roll_max_7_day,temp_roll_min_7_day,holiday_previous_day,holiday_next_day,temp,holiday
date_time,,,,,,,,,,,,,,,,,,,,,
2011-01-08 00:00:00,25.0,mist,0.5,0.866025,0.120537,0.992709,-0.781832,0.62349,0.258819,0.965926,...,8.063334,10.127976,9.02,6.56,18.86,4.92,0.0,0.0,7.38,0.0
2011-01-08 01:00:00,16.0,mist,0.5,0.866025,0.120537,0.992709,-0.781832,0.62349,0.500000,0.866025,...,8.029166,10.113334,9.02,6.56,18.86,4.92,0.0,0.0,7.38,0.0
2011-01-08 02:00:00,16.0,mist,0.5,0.866025,0.120537,0.992709,-0.781832,0.62349,0.707107,0.707107,...,7.995000,10.103572,9.02,6.56,18.86,4.92,0.0,0.0,7.38,0.0


In [3]:
# Data selection (reduce data size to speed up the example)
# ==============================================================================
data = data.drop(columns="weather")
data = data.loc["2012-01-01 00:00:00":]

## Create forecaster

A forecasting model is created to predict the number of users using the last 168 values (last week) and the exogenous features available in the dataset.

In [4]:
# Create forecaster
# ==============================================================================
forecaster = ForecasterAutoreg(
                 regressor = LGBMRegressor(
                                 n_estimators = 900,
                                 random_state = 15926,
                                 max_depth    = 7,
                                 verbose      = -1
                             ),
                 lags      = 168,
             )

## Feature selection with Recursive Feature Elimination (RFECV)


`select_features` function from the `skforecast.model_selection` module is used to select the best subset of features (lags and exogenous variables). This function is compatible with the feature selection methods implemented in the scikit-learn library. The following parameters are available:

- `forecaster`: Forecaster of type `ForecasterAutoreg` or `ForecasterAutoregCustom`.

- `selector`: Feature selector from `sklearn.feature_selection`. For example, `RFE` or `RFECV`.

- `y`: Target time series to which the feature selection will be applied.

- `exog`: Exogenous variable/s included as predictor/s

- `select_only_exog`: Whether to use only exogenous variables in the selection process. 
        
    + If `False` (default), lags are also evaluated by the selector and only those that are selected are included in the final list of selected features. 

    + If `True`, exogenous variables are evaluated without the presence of lags. Therefore, all lags are selected.

- `force_inclusion`: Features to force include in the final list of selected features.
        
    + If `list`, list of feature names to force include.
    
    + If `str`, regular expression to identify features to force include. For example, if `force_inclusion="^sun_"`, all features that begin with "sun_" will be included in the final list of selected features.

- `subsample`: Proportion of records to use for feature selection.

- `random_state`: Sets a seed for the random subsample so that the subsampling process is always deterministic.

- `verbose`: Print information about feature selection process.

### Selection of lags and exogenous features

By default, the `select_features` function selects the best subset of lags and exogenous variables.

In [5]:
# Feature selection (lags and exog) with scikit-learn RFECV
# ==============================================================================
regressor = LGBMRegressor(n_estimators=100, max_depth=5, random_state=15926, verbose=-1)

selector = RFECV(
    estimator=regressor, step=1, cv=3, min_features_to_select=25, n_jobs=-1
)

selected_lags, selected_exog = select_features(
    forecaster       = forecaster,
    selector         = selector,
    y                = data["users"],
    exog             = data.drop(columns="users"),
    select_only_exog = False,
    force_inclusion  = None,
    subsample        = 0.5,
    random_state     = 123,
    verbose          = True,
)

Recursive feature elimination (RFECV)
-------------------------------------
Total number of features available: 256
Total number of records available: 8592
Total number of records used for feature selection: 4296
Number of features selected: 53
    Selected lags (n=45): [1, 2, 3, 4, 5, 7, 9, 10, 11, 12, 14, 20, 23, 24, 25, 32, 52, 53, 59, 71, 86, 91, 96, 101, 104, 105, 108, 115, 120, 123, 131, 132, 143, 144, 145, 146, 150, 151, 152, 160, 163, 165, 166, 167, 168]
    Selected exog (n=8): ['hour_day_sin', 'hour_day_cos', 'poly_week_of_year_sin__hour_day_cos', 'poly_week_of_year_sin__sunrise_hour_cos', 'poly_week_day_cos__hour_day_cos', 'temp_roll_mean_1_day', 'temp_roll_mean_7_day', 'temp']


### Selection of exogenous features only

If `select_only_exog = True`, exogenous features are evaluated by the selector in the absence of lags (autoregressive variables). All lags are then added to the final list.

In [6]:
# Feature selection (only exog) with scikit-learn RFECV
# ==============================================================================
regressor = LGBMRegressor(n_estimators=100, max_depth=5, random_state=15926, verbose=-1)

selector = RFECV(
    estimator=regressor, step=1, cv=3, min_features_to_select=25, n_jobs=-1
)

selected_lags, selected_exog = select_features(
    forecaster       = forecaster,
    selector         = selector,
    y                = data["users"],
    exog             = data.drop(columns="users"),
    select_only_exog = True,
    subsample        = 0.5,
    verbose          = True,
)

Recursive feature elimination (RFECV)
-------------------------------------
Total number of features available: 88
Total number of records available: 8592
Total number of records used for feature selection: 4296
Number of features selected: 60
    Selected lags (n=168): [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 1

### Force selection of specific features

The `force_inclusion` argument can be used to force the selection of certain features. To illustrate this, a non-informative feature is added to the data set, `noise`. This feature contains no information about the target variable and therefore should not be selected by the feature selector. However, if we force the inclusion of this feature, it will be included in the final list of selected features.

In [5]:
# Add non-informative feature
# ==============================================================================
data['noise'] = np.random.normal(size=len(data))

In [8]:
# Feature selection (only exog) with scikit-learn RFECV
# ==============================================================================
regressor = LGBMRegressor(n_estimators=100, max_depth=5, random_state=15926, verbose=-1)

selector = RFECV(
    estimator=regressor, step=1, cv=3, min_features_to_select=10, n_jobs=-1
)

selected_lags, selected_exog = select_features(
    forecaster       = forecaster,
    selector         = selector,
    y                = data["users"],
    exog             = data.drop(columns="users"),
    select_only_exog = True,
    force_inclusion  = ["noise"],
    subsample        = 0.5,
    verbose          = True,
)

Recursive feature elimination (RFECV)
-------------------------------------
Total number of features available: 89
Total number of records available: 8592
Total number of records used for feature selection: 4296
Number of features selected: 64
    Selected lags (n=168): [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 1

In [9]:
# Check if "noise" is in selected_exog
# ==============================================================================
"noise" in selected_exog

True

## Feature selection with Sequential Feature Selection (SFS)

Sequential Feature Selection is a robust method for selecting features, but it is computationally expensive. When the data set is very large, one way to reduce the computational cost is to use a single validation split to evaluate each candidate model instead of cross-validation (default).

In [10]:
# Feature selection (only exog) with scikit-learn SequentialFeatureSelector
# ==============================================================================
regressor = LGBMRegressor(n_estimators=100, max_depth=5, random_state=15926, verbose=-1)

selector = SequentialFeatureSelector(
               estimator            = forecaster.regressor,
               n_features_to_select = 25,
               direction            = "forward",
               cv                   = ShuffleSplit(n_splits=1, test_size=0.3, random_state=951),
               scoring              = "neg_mean_absolute_error",
               n_jobs               = -1,
           )

selected_lags, selected_exog = select_features(
    forecaster       = forecaster,
    selector         = selector,
    y                = data["users"],
    exog             = data.drop(columns="users"),
    select_only_exog = True,
    subsample        = 0.2,
    random_state     = 123,
    verbose          = True,
)

Recursive feature elimination (SequentialFeatureSelector)
---------------------------------------------------------
Total number of features available: 89
Total number of records available: 8592
Total number of records used for feature selection: 1718
Number of features selected: 25
    Selected lags (n=168): [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 1

## Combination of feature selection methods

Combining feature selection methods can help speed up the process. An effective approach is to first use `SelectFromModel` to eliminate the less important features, and then use `SequentialFeatureSelector` to determine the best subset of features from this reduced list. This two-step method often improves efficiency by focusing on the most important features.

In [6]:
# Feature selection (lags and exog) with SelectFromModel + SequentialFeatureSelector
# ==============================================================================
regressor = LGBMRegressor(n_estimators=100, max_depth=5, random_state=15926, verbose=-1)

# Step 1: Select the 70% most important features with SelectFromModel
selector_1 = SelectFromModel(
                 estimator    = regressor,
                 max_features = int(data.shape[1] * 0.7),
                 threshold    = -np.inf
             )
selected_lags_1, selected_exog_1 = select_features(
    forecaster       = forecaster,
    selector         = selector_1,
    y                = data["users"],
    exog             = data.drop(columns="users"),
    select_only_exog = False,
    subsample        = 0.2,
    verbose          = True,
)

# Step 2: Select the 25 most important features with SequentialFeatureSelector
forecaster.set_lags(lags=selected_lags_1)
selector_2 = SequentialFeatureSelector(
                 estimator            = regressor,
                 n_features_to_select = 25,
                 direction            = "forward",
                 cv                   = ShuffleSplit(n_splits=1, test_size=0.3, random_state=951),
                 scoring              = "neg_mean_absolute_error",
                 n_jobs               = -1,
             )
selected_lags, selected_exog = select_features(
    forecaster       = forecaster,
    selector         = selector_2,
    y                = data["users"],
    exog             = data[selected_exog_1],
    select_only_exog = False,
    subsample        = 0.2,
    verbose          = True,
)

Recursive feature elimination (SelectFromModel)
-----------------------------------------------
Total number of features available: 257
Total number of records available: 8592
Total number of records used for feature selection: 1718
Number of features selected: 62
    Selected lags (n=55): [1, 2, 3, 4, 5, 6, 7, 9, 10, 13, 14, 16, 23, 24, 25, 26, 32, 35, 48, 49, 50, 52, 71, 72, 73, 86, 88, 95, 96, 102, 104, 107, 111, 120, 123, 124, 125, 127, 132, 133, 142, 143, 144, 145, 146, 147, 149, 150, 155, 159, 160, 161, 166, 167, 168]
    Selected exog (n=7): ['poly_week_of_year_cos__week_day_sin', 'poly_week_of_year_cos__sunrise_hour_cos', 'poly_week_day_sin__hour_day_sin', 'poly_hour_day_sin__hour_day_cos', 'temp_roll_mean_1_day', 'temp', 'noise']
Recursive feature elimination (SequentialFeatureSelector)
---------------------------------------------------------
Total number of features available: 62
Total number of records available: 8592
Total number of records used for feature selection: 1718